In [1]:
import urllib
import bs4
import pandas
from urllib import request
import numpy as np

In [7]:
urlfoot="https://fr.wikipedia.org/wiki/Liste_des_mouvements_litt%C3%A9raires"
request_text= request.urlopen(urlfoot).read()
page = bs4.BeautifulSoup(request_text, "lxml") #qui est bien une page html



# === On get la page d'intérêt ===
Le but de ce notebook est d'obtenir les données contenues dans les pages wikipédia des différents mouvements littéraires afin d'effectuer par la suite des prédictions associant des auteurs à leur mouvement.
On veut donc, dans la page wikipedia "Liste des mouvements littéraires" récupérer les liens correspondant à des mouvements littéraires, visiter ces pages et récuperer leur contenu.





In [9]:


test=page.findAll("ul")
L=[]
for res in test :
  test2 = res.findAll("li")
  L.append(test2)

print(L[15])

#On vérifier la forme des balises

[<li><a href="/wiki/Humanisme_de_la_Renaissance" title="Humanisme de la Renaissance">Humanisme de la Renaissance</a></li>, <li>La <a href="/wiki/Pl%C3%A9iade_(XVIe_si%C3%A8cle)" title="Pléiade (XVIe siècle)">Pléiade</a></li>, <li><a class="mw-redirect" href="/wiki/Picaresque" title="Picaresque">Picaresque</a></li>]


In [10]:
for res in L[15]:
    try :
       res2=res.find("a")
       print(type(res), res2.get("title"))    #https://stackoverflow.com/questions/57395509/get-item-from-bs4-element-tag
    except :
        pass

<class 'bs4.element.Tag'> Humanisme de la Renaissance
<class 'bs4.element.Tag'> Pléiade (XVIe siècle)
<class 'bs4.element.Tag'> Picaresque


Scrap

In [11]:



base=page.findAll("ul")         
L=[]
for ul in base :
  L.append(ul.findAll("li"))
#print(L)



In [12]:
resultat=[]
for i in range(len(L)) :      #On peut pas faire direct for li in L à cause du type
    for li in L[i]:
        try :
            inter=li.find("a") 
            if type(inter.get("title"))==str:
                s=inter.get('href')
                if s[:5]=='/wiki':
                    resultat.append(s)
      #print(type(inter))
        except:                   #On peut avoir type None pour inter 
            pass
len(resultat)

86

In [14]:
# resultat correspond à la liste des liens récupérés.

Nettoyage

In [19]:
res_mouvements = resultat[13:70]
# tous les liens de résultat ne correspondent pas à des mouvements littéraires. On regarde donc à la main où commence et où finissent les liens intéressants.

In [20]:
resultat[15][6:] #exemple de ce que l'on récupère

'Troubadour'

In [21]:
wiki_url = 'https://fr.wikipedia.org' #base de l'url wiki 

In [22]:
# on crée deux dictionnaires où l'on stocke les données intéressantes
# dico_bs stocke les pages html de chaque mouvement
# dico_txt stocke le texte 

dico_bs = {}
dico_txt = {}
for link in res_mouvements:
    url_ = wiki_url + link
    request_text= request.urlopen(url_).read()
    dico_bs[link[6:]]=bs4.BeautifulSoup(request_text, "lxml")
    txt=''
    page = bs4.BeautifulSoup(request_text, "lxml")
    base=page.findAll("p")         
    for h2 in base :
        for string in h2.strings:
            txt+=string
    dico_txt[link[6:]]=txt
   
    
    

In [4]:
#dico_txt['Troubadour'] exemple de résultat provisoire

In [27]:
BeautifulSoup = bs4.BeautifulSoup()

In [30]:
txt=''
base=test.findAll("p")         
for h2 in base :
    for string in h2.strings:
        txt+=string
   
        
    



AttributeError: ResultSet object has no attribute 'findAll'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [24]:
#txt
#L[5]

In [36]:

soup = BeautifulSoup(doc, "html.parser") 
  
tag = soup.body 
  
for string in tag.strings: 
   # print(string) 

SyntaxError: unexpected EOF while parsing (<ipython-input-36-f7a31e2fb7f3>, line 6)

In [31]:
len(dico_txt['Absurde'])

4682

In [ ]:
#tokenization
# dico de vocab pr chaque texte


In [33]:
#for cpl in dico_txt.items():
 #   key,value=cpl
  #  print(key),print(len(value))
  #Visualisation des résultats pour vérifier que les données récupérées ne sont pas aberrantes.

In [35]:
#dico_txt['Picaresque']

On va désormais traiter les données afin de les rendre exploitables. On va donc produire, à partir des textes bruts scrapés, produire des textes desquels seront retirés les "stopwords", c'est-à-dire les mots n'apportant pas de sens et de complexité au texte. On considère qu'ils sont inutiles car ne permettront pas de différencier les mouvements entre eux. On va également lemmatiser le texte, c'est-à-dire considérer la racine/le radical de chaque mot plutôt que le mot lui même, encore une fois pour nous intéresser au sens avant tout (on considère, par exemple,)

In [37]:
import spacy
import nltk
nlp = spacy.load("fr_core_news_sm")
from nltk.corpus import stopwords
nltk.download('stopwords')
stop=stopwords.words('french')


OSError: [E050] Can't find model 'fr_core_news_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [38]:
def clean_txt(s):
    doc = nlp(s)
    L=[]
    for token in doc: 
        s_t = str(token) #on récupère le token
        alphanum=True # on initialise cette variable booléenne
        if not s_t in stop:
            for char in s_t:
                alphanum = char.isalnum() and alphanum 
            if alphanum:            #si chaque char du token est alphanumérique, on le recupère
                L.append(token.lemma_) # et on ajoute sa forme lemmatisée
    L=np.array(L)
    return ' '.join(L) # on récupère le texte en espaçant les lemmes de ' '
        
    

In [41]:
dico_txt_clean={}
for cpl in dico_txt.items():
    key,value=cpl
    #print(key),print(len(value))
    dico_txt_clean[key]=clean_txt(value)

    #On récupère le texte "cleaned"

Litt%C3%A9rature_hagiographique
11557
Chanson_de_geste
12214
Troubadour
15412
Roman_courtois
3204
Fabliau
8626
Grands_rh%C3%A9toriqueurs
5366
Po%C3%A9sie_courtoise
1716
Humanisme_de_la_Renaissance
29058
Pl%C3%A9iade_(XVIe_si%C3%A8cle)
8652
Picaresque
5092
Litt%C3%A9rature_baroque
8374
Burlesque
7865
Classicisme
24412
Qui%C3%A9tisme
8922
Pr%C3%A9ciosit%C3%A9
3389
Moralisme
20133
Illuminisme
15969
Roman_gothique
8076
Pr%C3%A9romantisme
3174
Libertin
9416
Lumi%C3%A8res_(philosophie)
39936
Romantisme
18512
R%C3%A9alisme_(litt%C3%A9rature)
17681
Naturalisme_(litt%C3%A9rature)
6691
Parnasse_(litt%C3%A9rature)
9562
Symbolisme_(art)
51191
%C3%89cole_romane
2044
D%C3%A9cadentisme
9184
Esth%C3%A9tisme
785
Absurde
4682
Surr%C3%A9alisme
25045
Oulipo
9823
Futurisme
11262
Courant_de_conscience
4054
Dada%C3%AFsme
20677
Populisme_(litt%C3%A9rature)
1228
R%C3%A9alisme_socialiste_en_France
14123
Renaissance_litt%C3%A9raire_catholique_en_France
5652
Litt%C3%A9rature_prol%C3%A9tarienne
17017
Roman_jdanovi

In [39]:
test =dico_txt_clean['Absurde']
#test

NameError: name 'dico_txt_clean' is not defined

In [40]:
def tf(word,doc):
    L=doc.split()
    c=0
    n=len(L)
    for k in range(n):
        if L[k]==word:
            c+=1
    return c/n

In [41]:
from math import log

In [42]:
def idf(word,corpus):
    n=len(corpus)
    c=0
    for couple in corpus.items():
        key,value=couple
        if word in value:
            c+=1
    return log(n/c)
    

In [43]:
def tf_idf(word,doc,corpus):
    return tf(word,doc)*idf(word,corpus)

    #la mesure tf-idf permet de faire ressortir les mots signficatifs. Ici, on regarde, pour chaque mot, sa valeur relative dans le document (qui sera le contenu texte de la page wikipédia du mouvement littéraire) par rapport au corpus (l'ensemble des pages des mouvements littéraires).

In [66]:
tf_idf('absurde',test,dico_txt_clean) #exemple d'utilisation

0.09315690201130325

In [47]:
def n_important_words(n,doc,corpus):
    L=doc.split()
    m=len(L)
    M=[]
    vocab=[]
    for k in range(m):
        if not L[k] in vocab:
            M.append((tf_idf(L[k],doc,corpus),k))
            vocab.append(L[k])
    M.sort()
    M.reverse()
    return [L[M[k][1]] for k in range(n)] 
    #Le but est de faire ressortir les n mots les plus importants pour la mesure tf-idf pour chaque mouvement, relativement aux autres.
    # On considère que ces mots permettent de faire ressortir l'essence du mouvement. 

In [45]:
n_important_words(10,test,dico_txt_clean) #un exemple
#Pour l'absurde, les mots représentatifs sont les suivants. 

NameError: name 'n_important_words' is not defined

In [ ]:
import pandas as pd
Mouvements = pd.DataFrame(columns=['mouvement','texte','texte_lem','tf_idf'])
L=list(dico_txt.keys())

Mouvements['mouvement']=L
Mouvements['texte'] = [dico_txt[mov] for mov in L]
Mouvements['texte_lem']=[dico_txt_clean[mov] for mov in L]
Mouvements['tf_idf']=[n_important_words(50,dico_txt_clean[mov],dico_txt_clean) for mov in L]





On obtient finalement le dataframe Mouvements_wiki_cleaned.csv qui contient les mouvements littéraires scrapés, les textes scrapés du contenu de chaque page, le "cleaning" de chaque texte (cleaning et lemmatisation), et les 50 mots avec les plus gros scores tf-idf. 

Les pages wikipedia contenant cependant des textes inutiles "Cette page wikipédia est incomplète" ou "Pour les articles homonymes, voir...". Il faut donc retirer ces parties sans intérêt. Cela a été fait à la main.

In [ ]:

#for k in range(57):
 #   print(Mouvements['texte'][k][:100],k)

 #Pour chaque mouvement, on affiche donc le texte et on essaie de le découper au bon endroit.

 # Mouvements['texte']=Mouvements['texte'].apply(lambda texte: texte[54:])

In [ ]:
# Mouvements['texte']=Mouvements['texte'].apply(lambda s: s.replace('\n',''))

# Mouvements['texte']=Mouvements['texte'].apply(lambda s: s.replace('\'',''))

#autres traitements


In [ ]:
# Mouvements.to_csv('Mouvements_wiki_cleaned.csv')